In [ ]:
from ydata_profiling import ProfileReport
import os
from sklearn.preprocessing import LabelEncoder
import wandb
import pandas as pd 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error , r2_score
import numpy as np 
import joblib

In [9]:
run = wandb.init(project="predict_house_price", job_type="download_pca_data")

artifact = run.use_artifact('pca_X_y_train_test:latest', type='processed_data')

artifact_dir = artifact.download()

X_train = pd.read_csv(os.path.join(artifact_dir, "X_train.csv"), index_col=0)
X_test = pd.read_csv(os.path.join(artifact_dir, "X_test.csv"), index_col=0)
y_train = pd.read_csv(os.path.join(artifact_dir, "y_train.csv"), index_col=0)
y_test = pd.read_csv(os.path.join(artifact_dir, "y_test.csv"), index_col=0)

run.finish()

wandb:   4 of 4 files downloaded.  
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [10]:
# khởi tạo run
run = wandb.init(project="predict_house_price", job_type="model_evaluation")

# giả sử model của Kun là
model = xgb.XGBRegressor(
    n_estimators=100,    # số lượng cây
    learning_rate=0.01,   # tốc độ học
    max_depth=6,         # độ sâu tối đa mỗi cây
    random_state=42
)

model.fit(X_train, y_train)  # Train model

# lưu model ra file
joblib.dump(model, "xgb_model.pkl")

# tạo artifact
artifact = wandb.Artifact(name="xgb_model", type="model")
artifact.add_file("xgb_model.pkl")

# log artifact
run.log_artifact(artifact)

# kết thúc run
run.finish()


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [11]:
y_pred = model.predict(X_test) 

In [12]:
# 5. Đánh giá kết quả
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print("✅ RMSE:", round(rmse, 2))
print("📊 R² Score:", round(r2, 4))

✅ RMSE: 47280.17
📊 R² Score: 0.6292


In [13]:
run = wandb.init(project="predict_house_price", job_type="model_evaluation")

wandb.log({
    "RMSE": rmse,
    "y_pred": y_pred.tolist()  # Chuyển đổi y_pred sang list nếu cần (WandB không hỗ trợ numpy array trực tiếp)
})

run.finish()


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


RMSE,▁
RMSE,47280.17056
